# A análise de componentes principais 🎯

Esse notebook é um estudo sobre PCA no qual se debruçará sobre um estudo de caso acerca de dados da cidade de São Paulo, o qual busca estabelecer um ranking acerca de qual distrito apresenta mais promoção à vida.

### Breve Introdução :

Técnica responsável por realizar a transformação de dados, de modo a reduzir a dimensionalidade dos dados. Além disso, ao reduzir a dimensionalidade dos dados, consegue-se reduzir também a multicolinearidade, aspecto no qual um dado apresenta grande correlação com o outro, produzindo redundância.

### Funcionamento :

O PCA funciona a partir da junção de quatro técnicas, que são:

- Realizar a subtração dos dados de uma dimensão, com base em sua média, buscando com isso captar a sua variação;

- Realizar o calculo da matriz de covariância, para identificar quais dimensões apresentam similaridade no quanto variam - se variam ou não juntas. Consiste de uma técnica fundamental para a criação dos autovetores e autovalores;

- Autovetores e autovalores : autovetores apontam, no conjunto de dados, para a direação de maior variação nos dados, enquanto que os autovalores são a sua magnitude.

>

**Em síntese:**

Os componentes principais, então, são os autovetores ordenados com base em seus autovalores, de modo que o primeiro tende a ter comportamento ou caráter mais representativo, enquanto que o segundo menos em relação ao primeiro e assim por diante.

### Dicionário do dataframe

- cod_ibge:

Este é o código do Instituto Brasileiro de Geografia e Estatística (IBGE) que identifica cada município.
distritos: Número de distritos dentro do município.
renda: Renda média da população do município, geralmente medida em reais.

>

- quota:

Este campo pode ter diferentes interpretações dependendo do contexto, mas provavelmente refere-se a algum tipo de índice ou proporção. Seria útil saber a descrição completa da variável para te dar uma resposta mais precisa.

>

- escolaridade:

Nível médio de escolaridade da população do município.
idade: Idade média da população do município.

>

- mortalidade:

Taxa de mortalidade do município, geralmente medida por número de mortes por 1000 habitantes.

>

- txcresc:

Taxa de crescimento populacional do município, medida em percentual.

>

- causasext:

Número de mortes por causas externas no município (acidentes, homicídios, etc.).

>

- favel:

Proporção da população do município que vive em favelas.
denspop: Densidade populacional do município, medida em habitantes por km².

### Bibliotecas Utilizadas 📚

In [ ]:
!pip install factor_analyzer

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from factor_analyzer import calculate_kmo, calculate_bartlett_sphericity

In [ ]:
df = pd.read_csv('/content/distritos_sp.csv')

df.head()

,cod_ibge,distritos,renda,quota,escolaridade,idade,mortalidade,txcresc,causasext,favel,denspop
0,1,Água Rasa,1961,34.619999,7.6,32,13.86,-1.840000,52.980000,0.00,125.610001
1,12,Alto de Pinheiros,4180,75.959999,8.4,33,8.68,-2.520000,38.570000,0.69,57.560001
2,23,Anhanguera,1093,4.500000,5.8,23,15.36,18.120001,22.680000,0.00,8.570000
3,34,Aricanduva,1311,21.020000,6.8,27,18.43,-1.070000,76.220001,5.38,138.539993
4,45,Artur Alvim,1248,15.910000,7.0,27,19.73,-1.400000,67.250000,4.11,167.399994


Não há dados nulos, não disponíveis ou nulos:

In [ ]:
df.isna().sum()

cod_ibge        0
distritos       0
renda           0
quota           0
escolaridade    0
idade           0
mortalidade     0
txcresc         0
causasext       0
favel           0
denspop         0
dtype: int64

In [ ]:
df.duplicated().sum()

0

In [ ]:
df.isnull().sum()

cod_ibge        0
distritos       0
renda           0
quota           0
escolaridade    0
idade           0
mortalidade     0
txcresc         0
causasext       0
favel           0
denspop         0
dtype: int64

In [ ]:
df.columns

Index(['cod_ibge', 'distritos', 'renda', 'quota', 'escolaridade', 'idade',
       'mortalidade', 'txcresc', 'causasext', 'favel', 'denspop'],
      dtype='object')

### Criando um novo dataframe, utilizando o PCA

**OBS:**

Importante dizer que o PCA trabalha apenas com variáveis numéricas e, portanto, não com dados categóricos, 'strings', palavras, o que significa que para a sua realização serão no primeiro momento excluídas, podendo ser chamadas novamente para a formação de um dataframe já fatorizado (termo que estou utilizando como referência a dizer que ele já passara pela transformação do PCA, pois os componentes principais que formam as colunas são também denominados fatores).

In [ ]:
df_2 = df.drop(['cod_ibge', 'distritos'], axis = 1)

In [ ]:
df_2.head()

,renda,quota,escolaridade,idade,mortalidade,txcresc,causasext,favel,denspop
0,1961,34.619999,7.6,32,13.86,-1.840000,52.980000,0.00,125.610001
1,4180,75.959999,8.4,33,8.68,-2.520000,38.570000,0.69,57.560001
2,1093,4.500000,5.8,23,15.36,18.120001,22.680000,0.00,8.570000
3,1311,21.020000,6.8,27,18.43,-1.070000,76.220001,5.38,138.539993
4,1248,15.910000,7.0,27,19.73,-1.400000,67.250000,4.11,167.399994


### Critérios para se analisar se podemos efetuar ou não o PCA

Ainda que apresentamos um dataframe que tenha pouca dimensionalidade, sendo de apenas 9, em comparação ao que é visto comumente, devemos antes de realizar uma transformação dos dados mediante ao PCA responder a pergunta: esses dados estão dispostos de modo que possam ser, de fato, submetidos ao PCA ?

Para responder a essa pergunta, há duas abordagens, que é o critério de KMO e a esfericidade de Bartlett (extremamente necessária para avaliar se o dataframe pode ou não ser aplicado ao PCA).

>

**KMO:**

Avalia a proporção da variância compartilhada entre as variáveis, medindo a adequação do dataframe à análise fatorial, o que inclui o PCA.

- Interpretação :

Possui um intervalo de 0 a 1, de modo que os valores próximos a 1 indicam que o conjunto de dados são adequados para serem passados ao PCA, enquanto que valores abaixo, normalmente inferior a 0.6, indicam o oposto.

>

**Esfericidade de Bartlett:**

Teste de hipótese que verifica se a matriz de correlação das suas variáveis é significativamente (com significância estatística) diferente de uma matriz de identidade, a qual implica que não há uma correlação entre as variáveis, informando, portanto, que a transformação por meio do PCA não seria adequada.

Para a compreensão da hipótese, de que pode ser utilizada o PCA ou não, utilizamos da hipótese nula - Ho - (hipótese que pressupõe que que a matriz de correlação é equivalente à matriz de identidade) e a hipótese alternativa - Ha -, com significância estatística de, geralmente, 5%. Nesse sentido se o cálculo da esfericidade de Bartlett resulta num valor inferior a 0.05, informa que a Ho deve ser rejeitada, de modo que considera-se que a alternativa.



In [ ]:
# Realizando os critérios:

matriz_corr = df_2.corr()

# Métricas kmo:
kmo_variaveis, kmo = calculate_kmo(matriz_corr)

# Esfericidade de Bartlett:
qui_quadrado, pvalor = calculate_bartlett_sphericity(matriz_corr)

print(f'Métrica KMO para cada coluna do dataframe : \n {kmo_variaveis} \n\n Métrica KMO para o conjunto total dos dados : \n {kmo} \n\n')

print(f' Valor do q-quadrado : {qui_quadrado.round(2)} \n Valor do p-value : {pvalor} \n')

Métrica KMO para cada coluna do dataframe : 
 [0.7355042  0.7801622  0.68300246 0.66335166 0.75703291 0.62242182
 0.72930105 0.87250207 0.68069643] 

 Métrica KMO para o conjunto total dos dados : 
 0.7179374989798737 


 Valor do q-quadrado : 246.8 
 Valor do p-value : 2.970758005748527e-33 



/usr/local/lib/python3.10/dist-packages/factor_analyzer/utils.py:244: UserWarning: The inverse of the variance-covariance matrix was calculated using the Moore-Penrose generalized matrix inversion, due to its determinant being at or very close to zero.
  warnings.warn(


In [ ]:
# Verificando se pvalor é menor que 5%:

pvalor < 0.05

True

Como é menor, rejeitamos Ho, o que significa dizer que a esfericidade de Bartlett mostra que o conjunto de dados presentes é adequado ao PCA.

### Normalização :    

Sabemos o que consiste a transformação dos dados via o PCA, bem como das métricas necessárias para verificar se um conjunto de dados pode ser ou não submetido a essa técnica, mas falta revisar algo, a normalização.

Uma vez que a técnica do PCA se relaciona com a variância dos dados, faz-se necessário colocá-los sujeito a uma similar escala de modo que, do contrário, não seja eficiente a redução da dimensionalidade. Em virtude disso, surge a normalização, que fixa os dados na média, na medida que esses variam segundo um desvio padrão.

Assim, compreende-se que antes de submeter os dados a um PCA é necessário que sejam escalonados numa determinada escala, geralmente, por sua robustez, com a normalização (Z-Score).

In [ ]:
# Normalizando os dados e aplicando
# a transformação do PCA, por meio do pipeline:

SEED = 22

pipeline = Pipeline([('scaler', StandardScaler()), ('PCA',
                                                    PCA(n_components= 9,
                                                        random_state=SEED))])

pca = pipeline.fit_transform(df_2)

pca_components = pipeline[1].components_

# Visualizando a nova matriz gerada após o PCA:

df_2_pca = pd.DataFrame(data = pca)

df_2_pca.head()

,0,1,2,3,4,5,6,7,8
0,1.519105,-0.761299,0.440370,0.256326,-0.134060,-0.202264,-0.177369,0.025619,-0.350088
1,3.985600,1.555514,0.313021,0.070926,0.100206,0.332763,0.655952,-0.105873,-0.055641
2,-2.681205,2.356295,3.872196,3.061874,1.039798,0.920633,-0.793794,0.194619,-0.111070
3,-0.574202,-1.061710,-0.214948,-0.121274,0.105649,-0.204103,-0.011002,0.042348,-0.100050
4,-0.457007,-1.688282,-0.110352,0.089602,0.452111,-0.437592,-0.037891,-0.168406,-0.013653


In [ ]:
# Verificando a taxa de variância explicada por coluna
# em relação a cada uma das colunas:

print(f' Variância explicada por coluna : \n {pipeline[1].explained_variance_.round(4)} \n')
print(f' Total de colunas que são explicadas : {pipeline[1].explained_variance_.sum().round(3)} \n')
print(f' Percentual de explicação do novo dataframe : {pipeline[1].explained_variance_ratio_.sum().round(3)}')

 Variância explicada por coluna : 
 [5.0082 1.1766 1.0068 0.7317 0.5349 0.4025 0.131  0.0658 0.0372] 

 Total de colunas que são explicadas : 9.095 

 Percentual de explicação do novo dataframe : 1.0


### Identificando os autovetores

In [ ]:
# Essa matriz informa os autovetores presentes na transformação
# dos dados para a sua forma redimensionada.

# Cada um desses autovetores se referem a uma dimensão, o qual
# se relaciona a cada uma das variáveis do conjunto de dados,
# como renda, mortalidade e afins, aplicando um peso a elas.

# Os valores que vemos nesse array são os pesos a elas atribuidos

pca_components

array([[ 0.37378561,  0.40454585,  0.43416448,  0.431286  , -0.29453553,
        -0.31298937, -0.29942197, -0.20536447,  0.0746924 ],
       [ 0.34194215,  0.20986443, -0.02279207, -0.06065282, -0.16369471,
         0.31272645, -0.04069338,  0.41039975, -0.73495343],
       [-0.17295765, -0.15501444, -0.00935731,  0.06348322,  0.17379496,
         0.27775581, -0.43911857, -0.68814007, -0.41015077],
       [-0.06339126, -0.13996332, -0.08458471, -0.12690816, -0.58019562,
         0.45311541, -0.49035151,  0.1587924 ,  0.38148578],
       [ 0.34200389,  0.27796072,  0.00388393, -0.03140668,  0.68113225,
         0.35010444, -0.28968662,  0.14639622,  0.33601377],
       [ 0.30718994,  0.21266882, -0.051763  , -0.06507346, -0.21449531,
         0.46702002,  0.58641913, -0.46772469,  0.16347059],
       [ 0.35731885,  0.14779821, -0.5314497 , -0.53511308, -0.08829316,
        -0.42300489, -0.21675972, -0.21903914, -0.00154526],
       [-0.56495425,  0.74089992, -0.33214191,  0.12879361, -0

In [ ]:
fatores = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9']

n_fatores = df_2_pca.shape[1]
autovalores = pipeline[1].explained_variance_ratio_ * n_fatores

resumo = pd.DataFrame({'Fator': fatores, 'Autovalor': autovalores,
                       'Variância explicada': pipeline[1].explained_variance_ratio_})

resumo

,Fator,Autovalor,Variância explicada
0,F1,4.956031,0.550670
1,F2,1.164338,0.129371
2,F3,0.996354,0.110706
3,F4,0.724067,0.080452
4,F5,0.529302,0.058811
5,F6,0.398324,0.044258
6,F7,0.129669,0.014408
7,F8,0.065066,0.007230
8,F9,0.036849,0.004094


### Carga Fatorial

A carga fatorial representa o coeficiente da correlação entre uma variável original e um fator, podendo ser positiva ou negativa, de modo que se próxima de 1 significa que são correlacionadas e se próxima de -1 que são inversamente correlacionadas.


In [ ]:
# Verificando os autovetores num dataframe:

colunas = [['renda', 'quota', 'escolaridade', 'idade',
            'mortalidade', 'txcres', 'causasext', 'favel', 'denspop']]

pd.DataFrame(pca_components, columns = colunas, index = [f'autovetor {i+1}' for i in range(9)])

,renda,quota,escolaridade,idade,mortalidade,txcres,causasext,favel,denspop
autovetor 1,0.373786,0.404546,0.434164,0.431286,-0.294536,-0.312989,-0.299422,-0.205364,0.074692
autovetor 2,0.341942,0.209864,-0.022792,-0.060653,-0.163695,0.312726,-0.040693,0.410400,-0.734953
autovetor 3,-0.172958,-0.155014,-0.009357,0.063483,0.173795,0.277756,-0.439119,-0.688140,-0.410151
autovetor 4,-0.063391,-0.139963,-0.084585,-0.126908,-0.580196,0.453115,-0.490352,0.158792,0.381486
autovetor 5,0.342004,0.277961,0.003884,-0.031407,0.681132,0.350104,-0.289687,0.146396,0.336014
autovetor 6,0.307190,0.212669,-0.051763,-0.065073,-0.214495,0.467020,0.586419,-0.467725,0.163471
autovetor 7,0.357319,0.147798,-0.531450,-0.535113,-0.088293,-0.423005,-0.216760,-0.219039,-0.001545
autovetor 8,-0.564954,0.740900,-0.332142,0.128794,-0.043236,0.046332,-0.022126,0.019462,-0.010178
autovetor 9,-0.226373,0.235400,0.638974,-0.694279,0.002605,-0.004163,-0.011863,-0.045723,-0.027849


In [ ]:
raiz_autovalores = np.sqrt(autovalores)

In [ ]:
# Obtendo as cargas fatoriais por meio dos autovetores e autovalores:

cargas_fatoriais = pd.DataFrame(pca_components.T * raiz_autovalores,
                                columns = fatores,
                                index = colunas)

cargas_fatoriais

,F1,F2,F3,F4,F5,F6,F7,F8,F9
renda,0.832127,0.368971,-0.172642,-0.053941,0.248819,0.193877,0.128669,-0.144109,-0.043455
quota,0.900606,0.226453,-0.154732,-0.119098,0.202225,0.134222,0.053221,0.188990,0.045188
escolaridade,0.966543,-0.024594,-0.009340,-0.071975,0.002826,-0.032669,-0.191373,-0.084723,0.122658
idade,0.960135,-0.065447,0.063367,-0.107989,-0.022849,-0.041070,-0.192692,0.032853,-0.133274
mortalidade,-0.655699,-0.176634,0.173478,-0.493701,0.495545,-0.135374,-0.031794,-0.011029,0.000500
txcres,-0.696781,0.337446,0.277249,0.385565,0.254712,0.294750,-0.152322,0.011818,-0.000799
causasext,-0.666578,-0.043910,-0.438317,-0.417250,-0.210756,0.370106,-0.078054,-0.005644,-0.002277
favel,-0.457185,0.442840,-0.686884,0.135120,0.106508,-0.295195,-0.078875,0.004964,-0.008777
denspop,0.166281,-0.793048,-0.409402,0.324614,0.244461,0.103171,-0.000556,-0.002596,-0.005346


Utilizando o critério de Kaiser, selecionamos apenas os fatores F1 e F2:

In [ ]:
resumo = cargas_fatoriais.copy()

resumo = resumo[['F1', 'F2']]

resumo['Comunalidade'] = (resumo ** 2).sum(axis = 1)

resumo

,F1,F2,Comunalidade
renda,0.832127,0.368971,0.828575
quota,0.900606,0.226453,0.862372
escolaridade,0.966543,-0.024594,0.934811
idade,0.960135,-0.065447,0.926143
mortalidade,-0.655699,-0.176634,0.461141
txcres,-0.696781,0.337446,0.599374
causasext,-0.666578,-0.043910,0.446254
favel,-0.457185,0.442840,0.405126
denspop,0.166281,-0.793048,0.656574
